# The architecture of a K8s cluster
A K8s cluster is based on a master-slave architecture. Typically, a K8s cluster comprises one or more control plane and worker nodes. A node can be a physical or virtual machine. The control plane manages the state of the cluster. Upon receiving user requests, the control plane assign workloads across the worker nodes. 

[kubectl](https://kubernetes.io/docs/reference/kubectl/) is a command line tool to communicate with a K8s cluster's control plan. You already used it while setting up the MLOps platform. For example, you ran `kubectl get nodes` to check how many nodes your cluster has. Let's do it once again.

In [2]:
!kubectl get nodes

NAME                    STATUS   ROLES           AGE   VERSION
kind-ep-control-plane   Ready    control-plane   28m   v1.24.0
kind-ep-worker          Ready    <none>          28m   v1.24.0
kind-ep-worker2         Ready    <none>          28m   v1.24.0


Your cluster should have three nodes, one control plane and two worker nodes. The values in the "AGE" column vary, depending on how long your cluster has been running. 

# K8s objects
As per K8s docs, objects are fundamental entities used to represent and manage various aspects of your containerized workloads and the cluster itself. K8s objects are defined using [YAML](https://www.freecodecamp.org/news/what-is-yaml-the-yml-file-format/) (or JSON) configuration files. We use kubectl to apply these configurations to the K8s cluster for deploying and managing containerized workloads in the cluster. Some common Kubernetes objects include:
- Pod: A Pod is the smallest unit of Kubernetes and contains one container(or more tightly coupled containers), along with specifications of how the container(s) run.
- Deployment: A Deployment updates pods to the desired state specified by users.
- Service: A Service enable network access to pods. it exposes a set of pods as a stable network endpoint. 
- Ingress: An Ingress routes external HTTP/HTTPs traffic to Services. 
- ConfigMap: A ConfigMap stores configuration data as key-value pairs and can be consumed by pods, e.g., as environment variables.
- Secret: Similar to ConfigMaps, Secrets store data in key-value pairs and can be consumed by Pods. Unlike ConfigMaps which store non-confidential data, Secrets store sensitive data, such as passwords and SSH keys, which can be encrypted by the control plane.
- Namespace: A Namespace is a virtual partition of a Kubernetes cluster. Other objects, such as Pods, Deployments, and Services, can be created in a Namespace.
- Custom Resource Definition: It allows users to define custom objects. For example, you'll see a custom object named InferenceService later in the course. An InferenceService represents an ML inference service, it specifies how an ML model should be served so that it can receive user requests and respond with predictions. 

# Example
Let's use an example to gain some hands-on experience with K8s objects and the use of kubectl. In this example, we use a very simple Web App (source code [here](./app/app.py)) that serves a single endpoint and returns a JSON response containing user account and password information. The App will first try to retrieve the information using environmental variables. If the environmental variables are not set, default values will be returned.
```python
@app.route("/")
def hello_world():
    user_info = {
        "user account": os.environ.get("USER_ACCOUNT", "default_user"),
        "password": os.environ.get("PASSWORD", "default_pwd")
    }
    return user_info
```
All the YAML files used in this example can be found from the "manifests" directory.

### Namespace
Let's first create a Namespace called "new-ns" where the example App will run. We use `kubectl apply -f <filename>` to create new objects or apply changes to objects in a K8s cluster.

In [3]:
!kubectl apply -f manifests/namespace.yaml

namespace/new-ns created


We use `kubectl get <object-type> <object-name>` to list one or more objects. If the object name is not given, all objects of the given object type will be listed. 

In [4]:
!kubectl get ns new-ns

# You'll see there is a namespace called "new-ns"

NAME     STATUS   AGE
new-ns   Active   3s


### Deployment
The Docker image of the example Web App was built and pushed to DockerHub (a Docker repository). Now, let's create a Deployment for the example Web App. 

In [5]:
!kubectl apply -f manifests/deployment.yaml

deployment.apps/example-app created


In [7]:
!kubectl -n new-ns get pods # -n is used to specify in which namespace we want to check objects

# After a while, you should see there is a pod running in the "new-ns" namespace

NAME                           READY   STATUS    RESTARTS   AGE
example-app-69cc8f5b77-w2mx6   1/1     Running   0          4m50s


### Service
We need to create a Service for the Deployment so that our containerized App running in a Pod can be accessible through an endpoint. 

In [8]:
!kubectl apply -f manifests/service.yaml

service/example-app-service created


Now, our App is accessible inside a K8s cluster. To access it from outside of the cluster, we can use `kubectl port-forward` to create a network connection between our local environment and the Service so that we can visit the App from our local environment. Run the following command in a terminal:
```bash
kubectl -n new-ns port-forward svc/example-app-service 8002:8081 
```

Navigate to [http://localhost:8002](http://localhost:8002) and you should see the user info printed. Notice that because we haven't set any environmental variables for our App, the default values are returned. 

### Ingress
Using `port-forward` is not very convenient, especially when we have multiple Apps. Next, let's create an Ingress for the Service.

In [9]:
!kubectl apply -f manifests/ingress.yaml

ingress.networking.k8s.io/example-app-ingress created


Similar to what you did in setup, you need to add the entry `<floating-ip-of-cPouta-VM> example-app.local` to /etc/hosts so that your local host knows that it should resolve the host name to the IP of your remote cPouta VM. You can do this by running the following command in your terminal:
```bash
echo '<floating-ip-of-cPouta-VM> example-app.local' | sudo tee -a  /etc/hosts
```

Now, you can see the similar output if you nagivate to [http://example-app.local](http://example-app.local). 

### ConfigMap and Secret
Next, let's create a ConfigMap where we defined a key-value pair, with the key "USER_ACCOUNT" and the value "user1":
```yaml
data:
  USER_ACCOUNT: "user1"
```

In [10]:
!kubectl apply -f manifests/configmap.yaml

configmap/useraccount-configmap created


Let's create a Secret where we defined a key-value pair, with the key "PASSWORD" and the value the base64 encoded version of "mypassword". The encoded string will be decoded in the cluster when it's referenced by a Pod. 
```yaml
data:
  PASSWORD: bXlwYXNzd29yZA== #mypassword
```
<details>
<summary> <i>More about the safety of Secrets. </i></summary>
Everyone can decode a base64 encoded text so some encryption mechanisms, such as <a href=https://github.com/bitnami-labs/sealed-secrets>SealSecret</a> and <a href=https://github.com/getsops/sops>SOPS</a>, need to be used in real production.
</details>

In [11]:
!kubectl apply -f manifests/secret.yaml

secret/userpwd-secret created


Let's then update our Deployment so that the Pod can source the environmental variables from the ConfigMap and Secret we just created. We don't need to delete the old Deployment but can just apply the new changes to it. Notice that the Deployment name and Namespace in deployment2.yaml is same as deployment.yaml so K8s knows which object it should update. Upon receiving the changes, the Deployment will terminate the old Pod and create a new one. 

In [16]:
!kubectl apply -f manifests/deployment2.yaml

deployment.apps/example-app created


Now, navigate to [http://example-app.local](http://example-app.local) again, you should see the user info has been changed. 

# Hints for debugging 
It's expectable that we'll make mistakes when configure K8s objects in YAML files, or our applications just don't work as we want. Below are some common commands that help us investigate into what is going on. 

### kubectl describe
This command provides detailed information about Kubernetes objects. For example, if you run `kubectl -n new-ns describe deployment example-app`, you'll see something like

<img src="./images/describe-deployment.png" />

There are a lot of information but we usually need to docus on the events (at the end) that show you what has happened to the object. In this screenshot, we can see the Deployment has created one Pod for our App. 

Similarly, if we describe a Pod using`kubectl -n new-ns describe pod <pod-name>`, we can also see the events of the Pod:

<img src="./images/describe-pod.png" />

Specifically, the Pod was assigned to a worker node. It then pulled the Docker image and created a container using the image. 

### kubectl logs
This command `kubectl -n new-ns logs <pod-name>` retrieves the logs generated by the container running in a Pod. In our example, the container logs is like

<img src="./images/container-logs.png" />

The logs show that the container (or the Pod) has received two requests. 

### Lens -- a dashboard for K8s (optional)
Lens is a K8s dashboard that shows the objects, their events, and logs of Pods in a central place. If you're using the provided OVA, Lens is pre-installed and you can find it from the left dock in your Desktop. Otherwise you can install it from [its website](https://docs.k8slens.dev/getting-started/install-lens/). 

You'll be required to activate the Lens Desktop App the first time you use it. During the process you'll be asked to choose a subscription. You can choose the free version (Lens Desktop Personal) as shown below:

<img src="./images/lens-activate.png" width=800 />

[This video](https://youtu.be/fjUT2uucIZk) shows the basic use of Lens. 


# Clean up
Finally, let's delete everything we created in the "new-ns" namespace. 

In [17]:
%%bash
kubectl delete -f manifests/deployment2.yaml
kubectl delete -f manifests/service.yaml
kubectl delete -f manifests/configmap.yaml
kubectl delete -f manifests/secret.yaml
kubectl delete -f manifests/ingress.yaml
kubectl delete -f manifests/namespace.yaml

deployment.apps "example-app" deleted
service "example-app-service" deleted
configmap "useraccount-configmap" deleted
secret "userpwd-secret" deleted
ingress.networking.k8s.io "example-app-ingress" deleted
namespace "new-ns" deleted
